In [1]:
import tensorcraft as tc
import numpy as np
import matplotlib.pyplot as plt


In [3]:
tree = tc.compile("""
# Distance computation
D[j,k] = (X[j,l] - C[k,l]) * (X[j,l] - C[k,l])

# Argmin
M[j] = 1000
M[j] = (D[j,k] < M[j]) * (D[j,k] - M[j])
Y[j] = (D[j,k] == M[j]) * (k - Y[j])

# Cluster update
U[k,l] = 0
B[l] = (Y[j] == l)
U[l,k] = X[j,k] * (Y[j] == l)
U[l,k] = U[l,k] / B[l]
""")

tree.children

[Tree(Token('RULE', 'tensorexp'), [Tree(Token('RULE', 'tensor'), [Token('TENSORVAR', 'D'), Tree(Token('RULE', 'multiidxop'), [Tree(Token('RULE', 'multiidxop'), [Tree(Token('RULE', 'idxop'), [Tree(Token('RULE', 'idxproduct'), [Tree('indexvar', [Token('INDEXVAR', 'j')])])])]), Tree(Token('RULE', 'idxop'), [Tree(Token('RULE', 'idxproduct'), [Tree('indexvar', [Token('INDEXVAR', 'k')])])])])]), Tree(Token('RULE', 'tensorop'), [Tree('prod', [Tree(Token('RULE', 'tensorproduct'), [Tree(Token('RULE', 'operand'), [Tree('sub', [Tree(Token('RULE', 'tensorop'), [Tree(Token('RULE', 'tensorproduct'), [Tree(Token('RULE', 'operand'), [Tree(Token('RULE', 'signed_tensor'), [Tree(Token('RULE', 'tensor'), [Token('TENSORVAR', 'X'), Tree(Token('RULE', 'multiidxop'), [Tree(Token('RULE', 'multiidxop'), [Tree(Token('RULE', 'idxop'), [Tree(Token('RULE', 'idxproduct'), [Tree('indexvar', [Token('INDEXVAR', 'j')])])])]), Tree(Token('RULE', 'idxop'), [Tree(Token('RULE', 'idxproduct'), [Tree('indexvar', [Token('INDEX